In [ ]:
import os
import pandas as pd

# Load large synthetic prices dataset generated by scripts/setup_database.py
csv_path = os.path.join('data', 'raw', 'prices.csv')
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"Expected CSV at {csv_path}. Run: python scripts/setup_database.py")

prices = pd.read_csv(csv_path, parse_dates=['date'])
print('Rows:', len(prices), 'Cols:', len(prices.columns))
print('Date range:', prices['date'].min(), '→', prices['date'].max())
print('Symbols:', prices['symbol'].nunique())

# Quick sanity checks
print(prices.dtypes)
print(prices.head(5))


In [ ]:
# Overview statistics
import numpy as np

num_symbols = prices['symbol'].nunique()
num_days = prices['date'].nunique()
rows = len(prices)
print('Symbols:', num_symbols)
print('Trading days:', num_days)
print('Total rows:', rows)
print('Price range:', prices['close'].min(), '→', prices['close'].max())
print('Volume range:', prices['volume'].min(), '→', prices['volume'].max())


In [ ]:
# Missingness profile
missing_counts = prices.isnull().sum().sort_values(ascending=False)
missing_ratio = (missing_counts / len(prices)).round(4)
missing_df = (
    missing_ratio.rename('missing_ratio')
    .to_frame()
    .reset_index()
    .rename(columns={'index': 'column'})
)
print(missing_df.head(20))


In [ ]:
# Distribution snapshots
quantiles = prices[['open','high','low','close','volume']].quantile([0.01,0.1,0.5,0.9,0.99])
print(quantiles)

# Simple hist-like bins without plotting (counts per bin)
close_bins = pd.cut(prices['close'], bins=10)
print(close_bins.value_counts().sort_index().head(10))


In [ ]:
# Correlation matrix for numeric fields
num_df = prices[['open','high','low','close','volume','delivery_qty','delivery_percentage']].copy()
num_df = num_df.apply(pd.to_numeric, errors='coerce')
corr = num_df.corr().round(3)
print(corr)

# Top correlations by absolute value (excluding self)
stacked = corr.stack().reset_index()
stacked.columns = ['col1','col2','corr']
stacked = stacked[stacked['col1'] < stacked['col2']]
print(stacked.sort_values('corr', ascending=False).head(10))


In [ ]:
# Per-symbol sample slice
sample_symbol = prices['symbol'].iloc[0]
ps = prices[prices['symbol'] == sample_symbol].sort_values('date').copy()
ps['return'] = ps['close'].pct_change()
print('Symbol:', sample_symbol, 'rows:', len(ps))
print(ps.head(10))
print(ps.tail(10))


In [ ]:
# Rolling stats (20-day)
ps['roll_mean_20'] = ps['close'].rolling(20).mean()
ps['roll_std_20'] = ps['close'].rolling(20).std()
ps['zscore_20'] = (ps['close'] - ps['roll_mean_20']) / (ps['roll_std_20'] + 1e-6)
print(ps[['date','close','roll_mean_20','roll_std_20','zscore_20']].head(25))


In [ ]:
# Volatility snapshot
vol_df = (
    prices.sort_values(['symbol','date'])
    .groupby('symbol')['close']
    .pct_change()
    .dropna()
    .to_frame('ret')
)
vol_stats = vol_df['ret'].describe()
annualized_vol = vol_df['ret'].std() * (252 ** 0.5)
print('Return stats:', vol_stats.to_dict())
print('Annualized vol (approx):', annualized_vol)


In [ ]:
# Volume analysis
vol_by_symbol = prices.groupby('symbol')['volume'].agg(['mean','median','max','min']).reset_index()
print(vol_by_symbol.head(10))

# Daily aggregate volume
daily_vol = prices.groupby('date')['volume'].sum().reset_index().rename(columns={'volume':'total_volume'})
print(daily_vol.head(10))


In [ ]:
# Delivery percentage analysis
deliv = prices[['symbol','date','delivery_percentage']].dropna().copy()
deliv_stats = deliv['delivery_percentage'].describe()
print('Delivery % stats:', deliv_stats.to_dict())

by_symbol_deliv = deliv.groupby('symbol')['delivery_percentage'].mean().reset_index().rename(columns={'delivery_percentage':'avg_delivery_pct'})
print(by_symbol_deliv.head(10))


In [ ]:
# Seasonality / calendar effects
prices['day_of_week'] = prices['date'].dt.dayofweek
prices['month'] = prices['date'].dt.month
avg_ret_dow = (
    prices.sort_values(['symbol','date']).groupby(['symbol'])['close']
    .pct_change().groupby(prices['day_of_week']).mean()
)
print('Avg daily return by DOW:')
print(avg_ret_dow)

avg_vol_month = prices.groupby('month')['volume'].mean().round(0)
print('Avg volume by month:')
print(avg_vol_month)


In [ ]:
# Outlier detection (simple z-score rule)
returns = (
    prices.sort_values(['symbol','date']).groupby('symbol')['close']
    .pct_change()
)
z = (returns - returns.mean()) / (returns.std() + 1e-9)
outliers = prices.loc[z.abs() > 5].copy()
print('Outlier count:', len(outliers))
print(outliers[['symbol','date','close','volume']].head(20))


In [ ]:
# Summary text cell (programmatic print)
summary = []
summary.append(f"Rows: {len(prices)}; Symbols: {prices['symbol'].nunique()}")
summary.append(f"Dates: {prices['date'].min()} → {prices['date'].max()}")
summary.append(f"Close min→max: {prices['close'].min():.2f} → {prices['close'].max():.2f}")
summary.append(f"Missing columns (top 5): {prices.isnull().sum().sort_values(ascending=False).head(5).to_dict()}")
summary.append("EDA complete. Proceed to feature engineering.")
print("\n".join(summary))
